In [1]:
import argparse
import random
import math

import numpy as np
import pandas as pd
import scipy.sparse as sp

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init

from sklearn.preprocessing import StandardScaler

import dgl
from dgl.nn.pytorch import GraphConv
from dgl.nn.pytorch.conv import ChebConv


In [2]:
txt_f= "data/sensor_graph/graph_sensor_ids.txt"
with open(txt_f) as f:
    sensor_ids = f.read().strip().split(",")
    
print(sensor_ids)

['773869', '767541', '767542', '717447', '717446', '717445', '773062', '767620', '737529', '717816', '765604', '767471', '716339', '773906', '765273', '716331', '771667', '716337', '769953', '769402', '769403', '769819', '769405', '716941', '717578', '716960', '717804', '767572', '767573', '773012', '773013', '764424', '769388', '716328', '717819', '769941', '760987', '718204', '718045', '769418', '768066', '772140', '773927', '760024', '774012', '774011', '767609', '769359', '760650', '716956', '769831', '761604', '717495', '716554', '773953', '767470', '716955', '764949', '773954', '767366', '769444', '773939', '774067', '769443', '767750', '767751', '767610', '773880', '764766', '717497', '717490', '717491', '717492', '717493', '765176', '717498', '717499', '765171', '718064', '718066', '765164', '769431', '769430', '717610', '767053', '767621', '772596', '772597', '767350', '767351', '716571', '773023', '767585', '773024', '717483', '718379', '717481', '717480', '717486', '764120',

In [3]:
len(sensor_ids)

207

In [4]:
df = pd.read_csv("data/sensor_graph/distances_la_2012.csv", dtype={"from": "str", "to": "str"})
df
# df.info()

,from,to,cost
0,1201054,1201054,0.0
1,1201054,1201066,2610.9
2,1201054,1201076,2822.7
3,1201054,1201087,2911.5
4,1201054,1201100,7160.1
...,...,...,...
295369,825515,823663,9894.1
295370,825515,825494,475.6
295371,825515,825496,6458.6
295372,825515,825513,6934.2


In [5]:
df.head()

,from,to,cost
0,1201054,1201054,0.0
1,1201054,1201066,2610.9
2,1201054,1201076,2822.7
3,1201054,1201087,2911.5
4,1201054,1201100,7160.1


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 295374 entries, 0 to 295373
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   from    295374 non-null  object 
 1   to      295374 non-null  object 
 2   cost    295374 non-null  float64
dtypes: float64(1), object(2)
memory usage: 6.8+ MB


In [7]:
df.shape

(295374, 3)

In [8]:
def get_adjacency_matrix(distance_df, sensor_ids, normalized_k=0.1):
    """
    :param normalized_k: entries that become lower than normalized_k after normalization are set to zero for sparsity.
    :return: adjacency matrix
    """
    num_sensors = len(sensor_ids)
    dist_mx = np.zeros((num_sensors, num_sensors), dtype=np.float32)
    dist_mx[:] = np.inf
    print("\nDist_mx-->")
    print(dist_mx)
    
    # Builds sensor id to index map.
    sensor_id_to_ind = {}
#     print("enumerate(sensor_ids) ",enumerate(sensor_ids))
    for i, sensor_id in enumerate(sensor_ids):
        sensor_id_to_ind[sensor_id] = i
        
    print("\nsensor_id_to_ind-->")
    print(sensor_id_to_ind)
    
    # Fills cells in the matrix with distances.
    for row in distance_df.values:
        if row[0] not in sensor_id_to_ind or row[1] not in sensor_id_to_ind:
            continue
        else:
            dist_mx[sensor_id_to_ind[row[0]], sensor_id_to_ind[row[1]]] = row[2]
        
    print("\nDist_mx-->")
    print(dist_mx)

    # Calculates the standard deviation as theta.
    distances = dist_mx[~np.isinf(dist_mx)].flatten()
#     print(distances)
    std = distances.std()
#     print(std)
    adj_mx = np.exp(-np.square(dist_mx / std))
    print("\nadj_mx-->")
    print(adj_mx)
    # Make the adjacent matrix symmetric by taking the max.
    # adj_mx = np.maximum.reduce([adj_mx, adj_mx.T])

    # Sets entries that lower than a threshold, i.e., k, to zero for sparsity.
    adj_mx[adj_mx < normalized_k] = 0
    return adj_mx


In [9]:
adj_mx = get_adjacency_matrix(df, sensor_ids)


Dist_mx-->
[[inf inf inf ... inf inf inf]
 [inf inf inf ... inf inf inf]
 [inf inf inf ... inf inf inf]
 ...
 [inf inf inf ... inf inf inf]
 [inf inf inf ... inf inf inf]
 [inf inf inf ... inf inf inf]]

sensor_id_to_ind-->
{'773869': 0, '767541': 1, '767542': 2, '717447': 3, '717446': 4, '717445': 5, '773062': 6, '767620': 7, '737529': 8, '717816': 9, '765604': 10, '767471': 11, '716339': 12, '773906': 13, '765273': 14, '716331': 15, '771667': 16, '716337': 17, '769953': 18, '769402': 19, '769403': 20, '769819': 21, '769405': 22, '716941': 23, '717578': 24, '716960': 25, '717804': 26, '767572': 27, '767573': 28, '773012': 29, '773013': 30, '764424': 31, '769388': 32, '716328': 33, '717819': 34, '769941': 35, '760987': 36, '718204': 37, '718045': 38, '769418': 39, '768066': 40, '772140': 41, '773927': 42, '760024': 43, '774012': 44, '774011': 45, '767609': 46, '769359': 47, '760650': 48, '716956': 49, '769831': 50, '761604': 51, '717495': 52, '716554': 53, '773953': 54, '767470': 55, 

In [10]:
adj_mx

array([[1.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 1.       , 0.3909554, ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.7174379, 1.       , ..., 0.       , 0.       ,
        0.       ],
       ...,
       [0.       , 0.       , 0.       , ..., 1.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 1.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        1.       ]], dtype=float32)

In [11]:
sp_mx = sp.coo_matrix(adj_mx)

In [12]:
print(sp_mx)

  (0, 0)	1.0
  (0, 13)	0.22234692
  (0, 37)	0.5088465
  (0, 42)	0.13740046
  (0, 54)	0.4409315
  (0, 111)	0.21944807
  (0, 116)	0.41578582
  (0, 125)	0.10258747
  (0, 140)	0.11251228
  (0, 142)	0.7216228
  (0, 145)	0.87776124
  (0, 199)	0.119804084
  (1, 1)	1.0
  (1, 2)	0.3909554
  (1, 7)	0.3904571
  (1, 28)	0.573732
  (1, 78)	0.20909733
  (1, 107)	0.16216348
  (1, 177)	0.92621875
  (1, 183)	0.1548724
  (2, 1)	0.7174379
  (2, 2)	1.0
  (2, 28)	0.17432715
  (2, 49)	0.441055
  (2, 78)	0.92367977
  :	:
  (204, 82)	0.109405816
  (204, 89)	0.43908045
  (204, 130)	0.3428647
  (204, 146)	0.3038368
  (204, 197)	0.13277133
  (204, 204)	1.0
  (205, 69)	0.10102746
  (205, 72)	0.94447696
  (205, 113)	0.470247
  (205, 141)	0.26885048
  (205, 190)	0.99567246
  (205, 205)	1.0
  (206, 94)	0.13104253
  (206, 97)	0.5652668
  (206, 127)	0.9886928
  (206, 128)	0.39819893
  (206, 154)	0.5334241
  (206, 155)	0.9271291
  (206, 159)	0.78106517
  (206, 160)	0.41620797
  (206, 162)	0.14170425
  (206, 163)	0.6209

In [13]:
sp_mx.shape

(207, 207)

In [14]:
G = dgl.from_scipy(sp_mx)

In [15]:
print(G)

Graph(num_nodes=207, num_edges=1722,
      ndata_schemes={}
      edata_schemes={})


In [16]:
# st=set()
# for row in df.values:
#     st.add(row[1])
# print(len(st))

In [17]:
h5df = pd.read_hdf("data/metr-la.h5")
num_samples, num_nodes = h5df.shape
h5df.shape

(34272, 207)

In [18]:
scaler = StandardScaler()
h5df_data = scaler.fit_transform(h5df)
h5df_data.shape

(34272, 207)

In [19]:
h5df_data

array([[ 0.43077492,  0.44910511,  0.34941571, ...,  0.27924332,
         0.52828547,  0.50272961],
       [ 0.35524801,  0.50737407,  0.2576477 , ...,  0.07308336,
         0.4991686 ,  0.54981271],
       [ 0.41419584,  0.20646275, -0.03965034, ...,  0.40958413,
         0.57320864,  0.508615  ],
       ...,
       [ 0.54682847, -0.0492249 ,  0.48213805, ...,  0.52970212,
         0.44675824,  0.47199481],
       [ 0.53577575,  0.11253667,  0.28798423, ...,  0.5092565 ,
         0.46932381,  0.57923965],
       [ 0.46331904,  0.40301176,  0.33045538, ...,  0.54333253,
         0.51081535,  0.49815209]])

In [20]:
tsdata = h5df.to_numpy()
tsdata.shape

(34272, 207)

In [21]:
n_his= 144
n_pred= 5
n_route= num_nodes
blocks= [1, 16, 32, 64, 32, 128]
drop_prob = 0
num_layers= 9
batch_size = 50
epochs = 10
lr = 0.001

In [22]:
W = adj_mx
len_val = round(num_samples * 0.1)
len_train = round(num_samples * 0.7)
train = h5df[:len_train]
val = h5df[len_train : len_train + len_val]
test = h5df[len_train + len_val :]

In [23]:
train = scaler.fit_transform(train)
val = scaler.transform(val)
test = scaler.transform(test)

In [24]:
train.shape

(23990, 207)

In [25]:
val.shape

(3427, 207)

In [26]:
test.shape

(6855, 207)

In [27]:
device = (
    torch.device("cuda")
    if torch.cuda.is_available() and not ("store_true")
    else torch.device("cpu")
)

print(device)

cpu


In [28]:
def data_transform(data, n_his, n_pred, device):
    # produce data slices for training and testing
    n_route = data.shape[1]
    l = len(data)
    num = l - n_his - n_pred
    x = np.zeros([num, 1, n_his, n_route])
    print(x.shape)
    y = np.zeros([num, n_route])

    cnt = 0
    for i in range(l - n_his - n_pred):
        head = i
        tail = i + n_his
        x[cnt, :, :, :] = data[head:tail].reshape(1, n_his, n_route)
        y[cnt] = data[tail + n_pred - 1]
        cnt += 1
        
    print(torch.Tensor(x).to(device).shape, torch.Tensor(y).to(device).shape)
    return torch.Tensor(x).to(device), torch.Tensor(y).to(device)

In [29]:
x_train, y_train = data_transform(train, n_his, n_pred, device)
x_val, y_val = data_transform(val, n_his, n_pred, device)
x_test, y_test = data_transform(test, n_his, n_pred, device)

(23841, 1, 144, 207)
torch.Size([23841, 1, 144, 207]) torch.Size([23841, 207])
(3278, 1, 144, 207)
torch.Size([3278, 1, 144, 207]) torch.Size([3278, 207])
(6706, 1, 144, 207)
torch.Size([6706, 1, 144, 207]) torch.Size([6706, 207])


In [30]:
train_data = torch.utils.data.TensorDataset(x_train, y_train)
train_iter = torch.utils.data.DataLoader(train_data, batch_size, shuffle=True)
val_data = torch.utils.data.TensorDataset(x_val, y_val)
val_iter = torch.utils.data.DataLoader(val_data, batch_size)
test_data = torch.utils.data.TensorDataset(x_test, y_test)
test_iter = torch.utils.data.DataLoader(test_data, batch_size)

In [31]:
loss = nn.MSELoss()
print(loss)

MSELoss()


In [32]:
G = G.to(device)
print(G)

Graph(num_nodes=207, num_edges=1722,
      ndata_schemes={}
      edata_schemes={})


In [33]:
class TemporalConvLayer(nn.Module):
    """Temporal convolution layer.

    arguments
    ---------
    c_in : int
        The number of input channels (features)
    c_out : int
        The number of output channels (features)
    dia : int
        The dilation size
    """

    def __init__(self, c_in, c_out, dia=1):
        super(TemporalConvLayer, self).__init__()
        self.c_out = c_out
        self.c_in = c_in
        self.conv = nn.Conv2d(
            c_in, c_out, (2, 1), 1, dilation=dia, padding=(0, 0)
        )
#         print(self.conv)

    def forward(self, x):
        print("0")
        return torch.relu(self.conv(x))

In [34]:
class SpatioConvLayer(nn.Module):
    def __init__(self, c, Lk):  # c : hidden dimension Lk: graph matrix
        super(SpatioConvLayer, self).__init__()
        self.g = Lk
        self.gc = GraphConv(c, c, activation=F.relu)
#         print(self.g,"***",self.gc)

    def init(self):
        print("-1")
        stdv = 1.0 / math.sqrt(self.W.weight.size(1))
        self.W.weight.data.uniform_(-stdv, stdv)

    def forward(self, x):
        print("1")
        x = x.transpose(0, 3)
        x = x.transpose(1, 3)
        output = self.gc(self.g, x)
        output = output.transpose(1, 3)
        output = output.transpose(0, 3)
        return torch.relu(output)

In [35]:
class FullyConvLayer(nn.Module):
    def __init__(self, c):
        super(FullyConvLayer, self).__init__()
        self.conv = nn.Conv2d(c, 1, 1)
#         print("self.conv--> ",self.conv)

    def forward(self, x):
        print("2")
        return self.conv(x)

In [36]:
class OutputLayer(nn.Module):
    def __init__(self, c, T, n):
        super(OutputLayer, self).__init__()
        self.tconv1 = nn.Conv2d(c, c, (T, 1), 1, dilation=1, padding=(0, 0))
        self.ln = nn.LayerNorm([n, c])
        self.tconv2 = nn.Conv2d(c, c, (1, 1), 1, dilation=1, padding=(0, 0))
        self.fc = FullyConvLayer(c)

    def forward(self, x):
        print("3")
        x_t1 = self.tconv1(x)
        x_ln = self.ln(x_t1.permute(0, 2, 3, 1)).permute(0, 3, 1, 2)
        x_t2 = self.tconv2(x_ln)
        return self.fc(x_t2)

In [37]:
class STGCN_WAVE(nn.Module):
    def __init__(
        self, c, T, n, Lk, p, num_layers, device, control_str="TNTSTNTST"
    ):
        super(STGCN_WAVE, self).__init__()
        self.control_str = control_str  # model structure controller
        self.num_layers = len(control_str)
        self.layers = nn.ModuleList([])
        cnt = 0
        diapower = 0
        
        for i in range(self.num_layers):
            i_layer = control_str[i]
            if i_layer == "T":  # Temporal Layer
                self.layers.append(
                    TemporalConvLayer(c[cnt], c[cnt + 1], dia=2**diapower)
                )
                diapower += 1
                cnt += 1
            if i_layer == "S":  # Spatio Layer
                self.layers.append(SpatioConvLayer(c[cnt], Lk))
            if i_layer == "N":  # Norm Layer
                self.layers.append(nn.LayerNorm([n, c[cnt]]))
        self.output = OutputLayer(c[cnt], T + 1 - 2 ** (diapower), n)
        for layer in self.layers:
            layer = layer.to(device)
        
#         print("self.output--> ",self.layers)
#         print("self.output--> ",self.output)
        

    def forward(self, x):
        for i in range(self.num_layers):
            i_layer = self.control_str[i]
            if i_layer == "N":
                x = self.layers[i](x.permute(0, 2, 3, 1)).permute(0, 3, 1, 2)
            else:
                x = self.layers[i](x)
        return self.output(x)

In [38]:
model = STGCN_WAVE(
    blocks, n_his, n_route, G, drop_prob, num_layers, device, "TNTSTNTST"
).to(device)

print("model--> ",model)

model-->  STGCN_WAVE(
  (layers): ModuleList(
    (0): TemporalConvLayer(
      (conv): Conv2d(1, 16, kernel_size=(2, 1), stride=(1, 1))
    )
    (1): LayerNorm((207, 16), eps=1e-05, elementwise_affine=True)
    (2): TemporalConvLayer(
      (conv): Conv2d(16, 32, kernel_size=(2, 1), stride=(1, 1), dilation=(2, 2))
    )
    (3): SpatioConvLayer(
      (gc): GraphConv(in=32, out=32, normalization=both, activation=<function relu at 0x000002738BB05790>)
    )
    (4): TemporalConvLayer(
      (conv): Conv2d(32, 64, kernel_size=(2, 1), stride=(1, 1), dilation=(4, 4))
    )
    (5): LayerNorm((207, 64), eps=1e-05, elementwise_affine=True)
    (6): TemporalConvLayer(
      (conv): Conv2d(64, 32, kernel_size=(2, 1), stride=(1, 1), dilation=(8, 8))
    )
    (7): SpatioConvLayer(
      (gc): GraphConv(in=32, out=32, normalization=both, activation=<function relu at 0x000002738BB05790>)
    )
    (8): TemporalConvLayer(
      (conv): Conv2d(32, 128, kernel_size=(2, 1), stride=(1, 1), dilation=

In [39]:
optimizer = torch.optim.RMSprop(model.parameters(), lr=lr)
print(optimizer)

RMSprop (
Parameter Group 0
    alpha: 0.99
    centered: False
    differentiable: False
    eps: 1e-08
    foreach: None
    lr: 0.001
    maximize: False
    momentum: 0
    weight_decay: 0
)


In [40]:
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.7)
# print(scheduler)

In [41]:
# min_val_loss = np.inf
# for epoch in range(1, epochs + 1):
#     l_sum, n = 0.0, 0
#     model.train()
#     for x, y in train_iter:
#         y_pred = model(x).view(len(x), -1)
#         l = loss(y_pred, y)
#         optimizer.zero_grad()
#         l.backward()
#         optimizer.step()
#         l_sum += l.item() * y.shape[0]
#         n += y.shape[0]
    
#     print("print")
#     scheduler.step()
#     val_loss = evaluate_model(model, loss, val_iter)
#     if val_loss < min_val_loss:
#         min_val_loss = val_loss
#         torch.save(model.state_dict(), save_path)
#     print(
#         "epoch",
#         epoch,
#         ", train loss:",
#         l_sum / n,
#         ", validation loss:",
#         val_loss,
#     )

In [42]:
# best_model = STGCN_WAVE(
#     blocks, n_his, n_route, G, drop_prob, num_layers, device, "TNTSTNTST"
# ).to(device)

# print("best_model--> ",best_model)

In [43]:
# l = evaluate_model(model, loss, test_iter)
# MAE, MAPE, RMSE = evaluate_metric(model, test_iter, scaler)
# print("test loss:", l, "\nMAE:", MAE, ", MAPE:", MAPE, ", RMSE:", RMSE)

In [44]:
# l = evaluate_model(best_model, loss, test_iter)
# MAE, MAPE, RMSE = evaluate_metric(best_model, test_iter, scaler)
# print("test loss:", l, "\nMAE:", MAE, ", MAPE:", MAPE, ", RMSE:", RMSE)

In [45]:
# import pickle

In [46]:
# pickle.dump(best_model, open('best_model.pkl', 'wb'))

In [47]:
# pickle.dump(model, open('model.pkl', 'wb'))